<a href="https://colab.research.google.com/github/aviax1/AE2/blob/master/index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**set data and model**

In [ ]:
!pip install wandb

**set models**

In [35]:
import torch,wandb,os,warnings,csv
import torch.nn as nn
import torchvision.transforms as transforms
from torch.autograd import Variable
from tensorflow.keras.datasets import mnist
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

num_epochs=5           #
batch_size = 128       #
image_size=784         #
hidden_size=128        #
lv_size = 48           # Latent Variable 
learning_rate=1e-4     #
cret = nn.MSELoss()    # criterion
warnings.filterwarnings('ignore')

class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(image_size, hidden_size), 
            nn.ReLU(True), nn.Linear(hidden_size, hidden_size),
            nn.ReLU(True), nn.Linear(hidden_size, hidden_size),
            nn.ReLU(True), nn.Linear(hidden_size, hidden_size),
             nn.ReLU(True), nn.Linear(hidden_size, lv_size))
        self.decoder = nn.Sequential(
            nn.Linear(lv_size, hidden_size),nn.ReLU(True),
            nn.Linear(hidden_size, hidden_size),nn.ReLU(True),
            nn.Linear(hidden_size, hidden_size),nn.ReLU(True),
            nn.Linear(hidden_size, hidden_size),nn.ReLU(True),
             nn.Linear(hidden_size, image_size), nn.Tanh())

    def forward(self, x):
        return self.decoder(self.encoder(x))

def model_name(digit):
  return './ae_'+str(digit)+'.pth'

model = autoencoder()
tmodel=autoencoder()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-6)

class DigitDataSet(Dataset):
  def __init__(self, dataset):
      self.dataset = dataset
      self.transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize([0.5], [0.5])])
  def __len__(self):
      return len(self.dataset)
  def __getitem__(self, idx):
      if torch.is_tensor(idx):
          idx = idx.tolist()
      return self.transform( self.dataset[idx,:,:])


def save_model(digit,model):
  mn=model_name(digit)
  torch.save(model.state_dict(),mn )
  wandb.save(mn)
  print("save model "+ mn)

def load_model_ifexist(digit,model):
  mn=model_name(digit)
  if os.path.isfile(mn):
    model.load_state_dict(torch.load(mn))
    model.eval()
  return model

def train_by_digit(by_digit,dna,model,ne=num_epochs,opt=optimizer):
  dataloader = DataLoader(DigitDataSet(xtrain[dna==by_digit]), batch_size=batch_size,shuffle=True, num_workers=6)
  for epoch in range(ne):
    for data in dataloader:
      imgs = Variable(data.view(data.size(0), -1))
      output_imgs = model(imgs)
      loss = cret(output_imgs, imgs)
      opt.zero_grad()
      loss.backward()
      opt.step()
  return loss.data

**set dna**

In [27]:


def dna_init(n):
  arr = np.zeros( n,dtype=np.int32)
  for i in range(n):
    arr[i]=random.randint(0,9)
  return arr



(xtrain,ytrain), (xtest,ytest) = mnist.load_data()
dna = dna_init(len(xtrain))




[6 5 5 ... 3 1 1]
[5 0 4 ... 5 6 8]


In [26]:
def testscore(dna,ytrain):
  return len(dna[dna==ytrain])/len(ytrain)

testscore(dna,ytrain)

0.09956666666666666